# Analysing NanoVer recordings using MDAnalysis

The aim of this notebook is to demonstrate how recordings of simulations created using the recording feature in NanoVer can be analysed using MDAnalysis. We will use a pre-prepared recording of an OpenMM simulation of the methane + nanotube system examined in the [OpenMM nanotube tutorial](../openmm/openmm_nanotube.ipynb). If you're looking for a challenge, check out our [OpenMM polyalanine simulation notebook](../openmm/openmm_polyalanine.ipynb) first to learn how to record your own iMD-VR simulation to analyse!

We will specifically examine a pre-prepared trajectory produced during a simulation of the methane + nanotube system, in which a user interacts with the carbon atom of the methane to pull the methane around in space, through the nanotube in one direction, and then back through the nanotube in the other direction. 

## Visualise the recording (Optional)

If you don't want to visualise the recording, skip straight to the next section!

You can visualise this trajectory in VR by passing the recorded simulation as a `PlaybackSimulation` to an `OmniRunner` and connecting to the server. Full instructions on how to do this can be found in the [OpenMM polyalanine simulation notebook](../openmm/openmm_polyalanine.ipynb). To serve the recorded simulation, run the cell below: 

In [ ]:
from nanover.omni import OmniRunner
from nanover.omni.playback import PlaybackSimulation

nanover_recording = PlaybackSimulation(name='nanover_recording',
                                       traj='files/nanover_recording.traj',
                                       state='files/nanover_recording.state')
recording_runner = OmniRunner.with_basic_server(nanover_recording, name='nanotube-recording-server')
recording_runner.next()

Now you can connect to the recorded simulation in VR to visualise it. Note that as it is a recording of a simulation, you will not be able to interact with the system yourself.

When you have finished watching the recording, run the following cell to close the server:

In [ ]:
recording_runner.close()

## Loading the recorded trajectory

To analyse the trajectory using MDAnalysis, we first need to load it as an MDAnalysis universe. This can be achieved using functionality from the `nanover.mdanalysis` module. Let's import the relevant modules—while we're at it we'll import `numpy` and `matplotlib.pyplot`, as they'll come in handy for analysis.

In [ ]:
import MDAnalysis as mda

from nanover.mdanalysis import NanoverParser, NanoverReader
from nanover.mdanalysis.converter import mdanalysis_to_frame_data

import matplotlib.pyplot as plt
import numpy as np

Now we need to convert our recorded trajectory to an MDAnalysis universe. This is achieved using the `NanoverReader` and `NanoverParser` classes.

In [ ]:
# Define the path to the recorded trajectory
nanotube_trajectory = 'files/nanotube-recording.traj'

# Create the MDAnalysis universe from the NanoVer recorded trajectory
u = mda.Universe(
    nanotube_trajectory,
    format=NanoverReader,
    topology_format=NanoverParser,
)

Great, we have our methane + nanotube universe! Now we can use the functionality of MDAnalysis to retrieve the data from the recording.

In [ ]:
# Create empty lists to append simulation data to

frames = []                        # Frame indices
simulation_times = []              # Time elapsed in simulation time
times = []                         # Time elapsed in simulation time
timestamps = []                    # Timestamps for synchronising trajectory and state data

potential_energy = []              # Potential energy of the system
kinetic_energy = []                # Kinetic energy of the system

particle_positions = []            # Positions of the particles
particle_velocities = []           # Velocities of the particles
particle_forces_system = []        # Forces acting on the particles (excluding user forces)

user_energy = []                   # Potential energy of the user interaction
user_forces = []                   # iMD forces acting on the particles 

# Iterate through simulation frames and retrieve simulation data
for timestep in u.trajectory:
    #if timestep.frame != 0:
        frames.append(timestep.frame)
        times.append(timestep.time)
        simulation_times.append(timestep.data['system.simulation.time'])
        timestamps.append(timestep.data["elapsed"])
        
        potential_energy.append(timestep.data["energy.potential"])
        kinetic_energy.append(timestep.data["energy.kinetic"])
        
        particle_positions.append(u.atoms.positions)
        particle_velocities.append(u.atoms.velocities)
        particle_forces_system.append(u.atoms.forces)
        
        user_energy.append(timestep.data["energy.user.total"])
        user_forces.append(timestep.data["user_forces"])

# Next Steps

* Build your own [VR trajectory viewer](./mdanalysis_trajectory.ipynb) with MDAnalysis and NanoVer
* Dig into how [frames](../fundamentals/frame.ipynb) are constructed under the hood